In [55]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [56]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from MyEarlyStopping import MyEarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

## Training dataset (40%)

In [57]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [58]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)

In [59]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [60]:
subject = 4
train_image = 10 #10:20%, 20: 40%, 30:60%

In [61]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10.], dtype=float32)

## Validation dataset (20%)

In [62]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [63]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)


In [64]:
ALL_Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
ALL_Val_Inputs = np.array(ALL_Val_Inputs)

In [65]:
select_indexs_val = participant_class==subject
Val_Inputs = ALL_Val_Inputs[select_indexs_val]
Val_command_class = Val_command_class[select_indexs_val]
sum(Val_command_class)

array([10., 12., 10., 11., 11., 11., 11., 11., 11., 11.], dtype=float32)

## Test dataset (20%)

In [66]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [67]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)

In [68]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [69]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

In [70]:
# sum(Test_command_class_2)

## Speaker 6 Test dataset 

In [71]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [72]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [73]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [74]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [75]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [76]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [77]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [78]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [79]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [80]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Command_Acc_p15','Command_Acc_p1','Command_Acc_p2',
#                                      'Command_Acc_p3','Command_Acc_p4','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_single_model.csv')
# Perfomance
Perfomance                                                        

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
105,Subjet_4,13.972502,0.6611,0.6036,0.3796,0.6435,0.9908,0.69,0.27,0.216,0.2574,20
106,Subjet_4,17.836687,0.6280,0.5856,0.3611,0.5478,0.9817,0.67,0.27,0.200,0.2970,20
107,Subjet_4,12.951283,0.6262,0.5766,0.3426,0.5652,0.9725,0.68,0.29,0.216,0.2574,20
108,Subjet_4,13.637552,0.6593,0.5586,0.4167,0.6435,0.9817,0.70,0.33,0.208,0.3069,20


# Create model 

In [81]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Single model （size:10）

In [28]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_4",'Size':'10','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_4_model_size10_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_4_size10_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_4_size10_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
2/2 [==============================] - 2s 893ms/step - loss: 2.5311 - accuracy: 0.1200 - val_loss: 2.3196 - val_accuracy: 0.0917
Epoch 2/10
2/2 [==============================] - 1s 278ms/step - loss: 2.2676 - accuracy: 0.1000 - val_loss: 2.1896 - val_accuracy: 0.3028
Epoch 3/10
2/2 [==============================] - 1s 275ms/step - loss: 2.1400 - accuracy: 0.3600 - val_loss: 2.1118 - val_accuracy: 0.5780
Epoch 4/10
2/2 [==============================] - 1s 290ms/step - loss: 2.0708 - accuracy: 0.6100 - val_loss: 2.0568 - val_accuracy: 0.7523
Epoch 5/10
2/2 [==============================] - 1s 285ms/step - loss: 2.0154 - accuracy: 0.7800 - val_loss: 2.0023 - val_accuracy: 0.7615
Epoch 6/10
2/2 [==============================] - 1s 285ms/step - loss: 1.9583 - accuracy: 0.8000 - val_loss: 1.9603 - val_accuracy: 0.7982
Epoch 7/10
2/2 [==============================] - 1s 283ms/step - loss: 1.9191 - accuracy: 0.8800 - val_loss: 1.9214 - val_accuracy: 0.8440
Epoch 8/10
2/2 [====

2/2 [==============================] - 1s 284ms/step - loss: 1.8200 - accuracy: 0.7800 - val_loss: 1.8370 - val_accuracy: 0.8073
Epoch 7/10
2/2 [==============================] - 1s 282ms/step - loss: 1.7756 - accuracy: 0.8800 - val_loss: 1.7861 - val_accuracy: 0.8532
Epoch 8/10
2/2 [==============================] - 1s 296ms/step - loss: 1.7257 - accuracy: 0.9600 - val_loss: 1.7497 - val_accuracy: 0.8899
Epoch 9/10
2/2 [==============================] - 1s 299ms/step - loss: 1.6918 - accuracy: 0.9900 - val_loss: 1.7172 - val_accuracy: 0.9266
Epoch 10/10
2/2 [==============================] - 1s 309ms/step - loss: 1.6635 - accuracy: 1.0000 - val_loss: 1.6859 - val_accuracy: 0.9358
Epoch 1/10
2/2 [==============================] - 1s 320ms/step - loss: 1.6280 - accuracy: 1.0000 - val_loss: 1.6680 - val_accuracy: 0.9174
Epoch 2/10
2/2 [==============================] - 1s 282ms/step - loss: 1.6032 - accuracy: 1.0000 - val_loss: 1.6445 - val_accuracy: 0.9358
Epoch 3/10
2/2 [==============

In [29]:
resnet_model = tf.keras.models.load_model('Initial_subject_4_model_size10_0608.h5')
predictions = resnet_model.predict(Test_Inputs_4)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_4, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_4, axis=1)),4)
print(acc_c)

0.9725


In [30]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Subjet_4,13.672550,0.6501,0.5676,0.4352,0.6087,0.9633,0.68,0.36,0.288,0.2574,10
96,Subjet_4,14.037608,0.6225,0.6216,0.3241,0.5739,0.9817,0.61,0.30,0.168,0.2673,10
97,Subjet_4,11.904643,0.6206,0.5315,0.3796,0.5913,0.9725,0.63,0.22,0.280,0.2673,10
98,Subjet_4,14.447167,0.6188,0.5405,0.3889,0.5391,0.9725,0.66,0.24,0.176,0.2772,10


In [31]:
best_index

9

## Single model （size:20）

In [32]:
subject = 4
train_image = 20 #10:20%, 20: 40%, 30:60%

In [33]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([20., 20., 20., 20., 20., 20., 20., 20., 20., 20.], dtype=float32)

In [34]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_4",'Size':'20','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_4_model_size20_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_4_size20_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_4_size20_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
4/4 [==============================] - 2s 412ms/step - loss: 2.4794 - accuracy: 0.0800 - val_loss: 2.1829 - val_accuracy: 0.2018
Epoch 2/10
4/4 [==============================] - 1s 178ms/step - loss: 2.1176 - accuracy: 0.2500 - val_loss: 1.9994 - val_accuracy: 0.3761
Epoch 3/10
4/4 [==============================] - 1s 181ms/step - loss: 1.9528 - accuracy: 0.5250 - val_loss: 1.8920 - val_accuracy: 0.6422
Epoch 4/10
4/4 [==============================] - 1s 173ms/step - loss: 1.8584 - accuracy: 0.6650 - val_loss: 1.8225 - val_accuracy: 0.7798
Epoch 5/10
4/4 [==============================] - 1s 182ms/step - loss: 1.7878 - accuracy: 0.8600 - val_loss: 1.7650 - val_accuracy: 0.8716
Epoch 6/10
4/4 [==============================] - 0s 86ms/step - loss: 1.7364 - accuracy: 0.8900 - val_loss: 1.7246 - val_accuracy: 0.8716
Epoch 7/10
4/4 [==============================] - 0s 82ms/step - loss: 1.6912 - accuracy: 0.8950 - val_loss: 1.6797 - val_accuracy: 0.8624
Epoch 8/10
4/4 [======

Epoch 10/10
4/4 [==============================] - 1s 185ms/step - loss: 1.6067 - accuracy: 0.9950 - val_loss: 1.6086 - val_accuracy: 0.9633
Epoch 1/10
4/4 [==============================] - 1s 198ms/step - loss: 1.5678 - accuracy: 1.0000 - val_loss: 1.5723 - val_accuracy: 0.9541
Epoch 2/10
4/4 [==============================] - 0s 82ms/step - loss: 1.5288 - accuracy: 1.0000 - val_loss: 1.5396 - val_accuracy: 0.9541
Epoch 3/10
4/4 [==============================] - 1s 186ms/step - loss: 1.4981 - accuracy: 1.0000 - val_loss: 1.5106 - val_accuracy: 0.9633
Epoch 4/10
4/4 [==============================] - 1s 174ms/step - loss: 1.4631 - accuracy: 1.0000 - val_loss: 1.4703 - val_accuracy: 0.9725
Epoch 5/10
4/4 [==============================] - 1s 172ms/step - loss: 1.4259 - accuracy: 1.0000 - val_loss: 1.4361 - val_accuracy: 0.9817
Epoch 6/10
4/4 [==============================] - 0s 81ms/step - loss: 1.3903 - accuracy: 1.0000 - val_loss: 1.4019 - val_accuracy: 0.9817
Epoch 7/10
4/4 [=====

Epoch 5/10
4/4 [==============================] - 0s 82ms/step - loss: 1.3990 - accuracy: 1.0000 - val_loss: 1.4141 - val_accuracy: 0.9817
Epoch 6/10
4/4 [==============================] - 1s 168ms/step - loss: 1.3628 - accuracy: 1.0000 - val_loss: 1.3791 - val_accuracy: 0.9817


In [35]:
resnet_model = tf.keras.models.load_model('Initial_subject_4_model_size20_0608.h5')
predictions = resnet_model.predict(Test_Inputs_4)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_4, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_4, axis=1)),4)
print(acc_c)

0.9817


In [36]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
105,Subjet_4,13.972502,0.6611,0.6036,0.3796,0.6435,0.9908,0.69,0.27,0.216,0.2574,20
106,Subjet_4,17.836687,0.6280,0.5856,0.3611,0.5478,0.9817,0.67,0.27,0.200,0.2970,20
107,Subjet_4,12.951283,0.6262,0.5766,0.3426,0.5652,0.9725,0.68,0.29,0.216,0.2574,20
108,Subjet_4,13.637552,0.6593,0.5586,0.4167,0.6435,0.9817,0.70,0.33,0.208,0.3069,20


In [37]:
best_index

0

## Single model （size:30）

In [82]:
subject = 4
train_image = 30 #10:20%, 20: 40%, 30:60%

In [83]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([30., 30., 30., 30., 30., 30., 30., 30., 30., 30.], dtype=float32)

In [84]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_4",'Size':'30','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_4_model_size30_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_4_size30_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_4_size30_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
5/5 [==============================] - 2s 442ms/step - loss: 2.3163 - accuracy: 0.1633 - val_loss: 2.0559 - val_accuracy: 0.5505
Epoch 2/10
5/5 [==============================] - 1s 169ms/step - loss: 1.9767 - accuracy: 0.6600 - val_loss: 1.8933 - val_accuracy: 0.8440
Epoch 3/10
5/5 [==============================] - 1s 165ms/step - loss: 1.8435 - accuracy: 0.8833 - val_loss: 1.7927 - val_accuracy: 0.9450
Epoch 4/10
5/5 [==============================] - 1s 165ms/step - loss: 1.7490 - accuracy: 0.9867 - val_loss: 1.7256 - val_accuracy: 0.9725
Epoch 5/10
5/5 [==============================] - 0s 85ms/step - loss: 1.6783 - accuracy: 0.9900 - val_loss: 1.6547 - val_accuracy: 0.9725
Epoch 6/10
5/5 [==============================] - 0s 86ms/step - loss: 1.6107 - accuracy: 0.9933 - val_loss: 1.5950 - val_accuracy: 0.9725
Epoch 7/10
5/5 [==============================] - 1s 165ms/step - loss: 1.5494 - accuracy: 0.9967 - val_loss: 1.5422 - val_accuracy: 0.9817
Epoch 8/10
5/5 [======

Epoch 2/10
5/5 [==============================] - 0s 91ms/step - loss: 0.9544 - accuracy: 1.0000 - val_loss: 0.9529 - val_accuracy: 0.9817
Epoch 3/10
5/5 [==============================] - 0s 87ms/step - loss: 0.8700 - accuracy: 1.0000 - val_loss: 0.8747 - val_accuracy: 0.9817
Epoch 4/10
5/5 [==============================] - 0s 89ms/step - loss: 0.7971 - accuracy: 1.0000 - val_loss: 0.8050 - val_accuracy: 0.9817
Epoch 5/10
5/5 [==============================] - 0s 90ms/step - loss: 0.7323 - accuracy: 1.0000 - val_loss: 0.7436 - val_accuracy: 0.9817
Epoch 6/10
5/5 [==============================] - 1s 171ms/step - loss: 0.6715 - accuracy: 1.0000 - val_loss: 0.6878 - val_accuracy: 0.9817
Epoch 1/10
5/5 [==============================] - 2s 347ms/step - loss: 2.3525 - accuracy: 0.1633 - val_loss: 2.0880 - val_accuracy: 0.5046
Epoch 2/10
5/5 [==============================] - 1s 176ms/step - loss: 1.9933 - accuracy: 0.7100 - val_loss: 1.9142 - val_accuracy: 0.8807
Epoch 3/10
5/5 [========

5/5 [==============================] - 0s 87ms/step - loss: 1.2803 - accuracy: 0.9900 - val_loss: 1.2479 - val_accuracy: 0.9817
Epoch 6/10
5/5 [==============================] - 1s 182ms/step - loss: 1.1847 - accuracy: 0.9967 - val_loss: 1.1670 - val_accuracy: 0.9908
Epoch 7/10
5/5 [==============================] - 1s 110ms/step - loss: 1.0988 - accuracy: 0.9967 - val_loss: 1.0820 - val_accuracy: 0.9817
Epoch 8/10
5/5 [==============================] - 1s 103ms/step - loss: 1.0201 - accuracy: 1.0000 - val_loss: 1.0104 - val_accuracy: 0.9817
Epoch 9/10
5/5 [==============================] - 1s 145ms/step - loss: 0.9502 - accuracy: 1.0000 - val_loss: 0.9435 - val_accuracy: 0.9817
Epoch 10/10
5/5 [==============================] - 0s 100ms/step - loss: 0.8817 - accuracy: 1.0000 - val_loss: 0.8817 - val_accuracy: 0.9817
Epoch 1/10
5/5 [==============================] - 1s 182ms/step - loss: 0.8197 - accuracy: 1.0000 - val_loss: 0.8237 - val_accuracy: 0.9817
Epoch 2/10
5/5 [===============

In [85]:
resnet_model = tf.keras.models.load_model('Initial_subject_4_model_size30_0608.h5')
predictions = resnet_model.predict(Test_Inputs_4)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_4, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_4, axis=1)),4)
print(acc_c)

0.9908


In [86]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
115,Subjet_4,15.352048,0.6832,0.6126,0.5000,0.6435,0.9908,0.67,0.31,0.240,0.2970,30
116,Subjet_4,16.284204,0.6667,0.5676,0.3889,0.6348,0.9908,0.76,0.30,0.232,0.2673,30
117,Subjet_4,15.602407,0.6667,0.5676,0.4352,0.6087,0.9908,0.74,0.35,0.232,0.2772,30
118,Subjet_4,17.284093,0.7127,0.6396,0.5370,0.6522,0.9908,0.75,0.29,0.216,0.2772,30


In [87]:
best_index

8

In [88]:
Perfomance.to_csv('Performance_0608_single_model.csv',index=False)